https://python.langchain.com/docs/use_cases/graph/integrations/graph_networkx_qa#create-the-graph

In [14]:
# import libraries
from langchain.indexes import GraphIndexCreator
from langchain.indexes.graph import NetworkxEntityGraph
from langchain_community.llms import Ollama
from langchain_openai import OpenAI
from langchain.chains import GraphQAChain
import networkx as nx
import pandas as pd
from dotenv import load_dotenv

In [15]:
load_dotenv()

True

In [16]:
# read csv file and skip first row
highway_type = pd.read_csv('edge_labels_highways.csv', header=None, skiprows=1,
                 names=['edge', 'highway type'])
# split the first column into two columns
highway_type['edge'] = highway_type['edge'].apply(lambda x: eval(x))

# Split the 'edge' column into two columns 'start' and 'end'
highway_type[['startNode', 'endNode']] = highway_type['edge'].apply(pd.Series)
# convert the values into strings
highway_type['startNode'] = highway_type['startNode'].astype(str)
highway_type['endNode'] = highway_type['endNode'].astype(str)
# Drop the 'edge' column
highway_type.drop('edge', axis=1, inplace=True)

In [17]:
# create networkx graph based on csv files
G = nx.read_edgelist("..\\csv\\edges_UH_Graph.csv", delimiter=",", create_using=nx.DiGraph(), nodetype=str)
# add labels to edges according to highway
for u,v in G.edges():
    if (u,v) in zip(highway_type['startNode'], highway_type['endNode']):
        G[u][v]['relation'] = 'highway type: ' + str(highway_type['highway type'].loc[(highway_type['startNode'] == u) & (highway_type['endNode'] == v)].values[0])
    else:
        G[u][v]['relation'] = 'highway type: unknown'

In [18]:
# write the graph to a GML file
nx.write_gml(G, "UH_Graph.gml")

In [19]:
# read the graph from the GML file
G = nx.read_gml("UH_Graph.gml")

In [20]:
# load gml graph as NetworkxEntityGraph
G = nx.read_gml("UH_Graph.gml")
G_NX = NetworkxEntityGraph(G)

In [21]:
# print amount of nodes
print("Nodes:", G_NX.get_number_of_nodes())

Nodes: 221


In [22]:
# define the chain
chain = GraphQAChain.from_llm(OpenAI(), graph = G_NX, verbose = True)

In [23]:
node_1 = '31404364'
node_2 = '313157654'

In [24]:
prompt = 'Which is the highway type between node {node_1} and node {node_2} from the graph you received as input?'

In [25]:
# chain.run(prompt)

## Problem: This is created for Knowledge Graphs, not for our graph!!